In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np


In [2]:
def mplot(img,img2=None):
    
    cv2.namedWindow('img',cv2.WINDOW_NORMAL)
    cv2.imshow('img',img)
    if img2 is not None:
        cv2.namedWindow('img2',cv2.WINDOW_NORMAL)
        cv2.imshow('img2',img2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
'''
function to rearrange the contour bounding boxes. in default the contour bounding boxes comes in the sorted order of
their y co-ordinates . this function returns a list of rectangles [(x1,y1,w1,h1),(x2,y2,w2,h2)...] which are sorted in
the order of x axis on each line. a line will have all recangles of y coordinates between y and y+h of first rectangle '''
def rearrange(cnt):
    b_rect=[]
    for c in cnt:
        rect=cv2.boundingRect(c)
        if rect[2] <=20 or rect[3] <= 20:
            continue
        b_rect.append(rect)
        
    p=b_rect[0][1]+b_rect[0][3]
    #print('length of brect:',len(b_rect))
    s_rect=[]
    i=0
    length=len(b_rect)
    while i<length:
        p=b_rect[i][1]+b_rect[i][3]
        elem_on_line=[]#elements on a line
        outer=True
        while i<length and p>b_rect[i][1]:
            elem_on_line.append(b_rect[i])
            i+=1
            outer=False
        if outer:
            i+=1
        elem_on_line=sorted(elem_on_line) #,key=lambda x:x[0]
        #print(elem_on_line,i)
        s_rect.extend(elem_on_line)
    return s_rect
        

In [4]:
img=cv2.imread('/home/jab/main/realtest/img2.jpg',0)
img2=img.copy()

ellipse=(3,3)
rect=(2,2)
threshold=128
#  _, small = cv2.threshold( small, threshold, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, ellipse )
grad = cv2.morphologyEx( img, cv2.MORPH_GRADIENT, kernel)
_, bw = cv2.threshold(grad, threshold, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, rect )
connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)
#  imshow( connected ); import ipdb; ipdb.set_trace()
_,contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL ,cv2.CHAIN_APPROX_SIMPLE )
#reversing contour list to start processig from top
contours.reverse()
print(len(contours))

350


In [5]:
mplot(grad,connected)


In [8]:
'''clear_noice method clear noice from list of images and retrun cleared list of images
'''
def clear_noice(image,ellipse=(7,7),rect=(13,7)):
    
    threshold=128
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, ellipse )
    kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, rect )
    cnt=[]
    grad = cv2.morphologyEx( image, cv2.MORPH_GRADIENT, kernel)
    _, bw = cv2.threshold(grad, threshold, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel2)
    _,contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL ,cv2.CHAIN_APPROX_SIMPLE )
    contours.reverse()
    cnt.extend(contours)
    print('contour length:',len(contours))
    #mplot(grad,connected)
    return cnt

In [1]:
img=cv2.imread('/home/jab/main/realtest/img12.jpg',0)
img2=img.copy()
contours=clear_noice(img) # it expects a list of images so passing single image as list 
print(len(contours))

NameError: name 'cv2' is not defined

In [10]:
s_rect=rearrange(contours)
words=[]
i=0
print("length of rect",len(s_rect))
for rect in s_rect:
    x,y,w,h = rect
    cv2.rectangle(img2,(x,y),(x+w,y+h),(0,0,0),2)
    cv2.putText(img2,str(i),(x+w+10,y+h),0,0.3,(0,0,0))
    words.append(img[y-2:y+h+2,x-2:x+w+2])
    i+=1
mplot(img2)

length of rect 11


<h4> Drawing bounding boxes on each words</h4><br/>
<p> Here, word consists of list of words cropped.we are rearranging each word and drawing bounding rectangle to each characters  </p>

In [ ]:
for i in range(len(words)):
    c=clear_noice(words[i],ellipse=(3,3),rect=(2,2))
    s_rect=rearrange(c)
    for rect in s_rect:
        x,y,w,h = rect
        cv2.rectangle(words[i],(x,y),(x+w,y+h),(0,0,0),2)
        cv2.putText(words[i],str(i),(x+w+10,y+h),0,0.3,(0,0,0))
    mplot(words[i])
    

In [ ]:
r=s_rect[1]
x,y,w,h=r
roi=img[y:y+h,x:x+w]
#roi=cv2.resize(roi,(86,86))
mplot(roi)

In [ ]:
ret,thresh3 = cv2.threshold(roi,127,255,cv2.THRESH_TRUNC)
ret,thresh4 = cv2.threshold(roi,127,255,cv2.THRESH_TOZERO)
ret,thresh5 = cv2.threshold(roi,127,255,cv2.THRESH_TOZERO_INV)
mplot(thresh3,thresh4)


In [ ]:
#applying morphological operation on each word in order to seperate characters
ellipse=(3,3)
rect=(2,2)
threshold=128
#  _, small = cv2.threshold( small, threshold, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
ker = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, ellipse )
grad = cv2.morphologyEx( roi, cv2.MORPH_GRADIENT, ker)
_, bw = cv2.threshold(grad, threshold, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
ker = cv2.getStructuringElement(cv2.MORPH_RECT, rect )
connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, ker)
#  imshow( connected ); import ipdb; ipdb.set_trace()
_,cnt, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL ,cv2.CHAIN_APPROX_SIMPLE )
#reversing contour list to start processig from top
cnt.reverse()
#cn = cv2.erode(connected, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2) ), iterations=2)

In [ ]:
mplot(bw,connected)

In [ ]:
s_rect=rearrange(contours)
i=0
print("length of rect",len(s_rect))
for rect in s_rect:
    x,y,w,h = rect
    cv2.rectangle(roi,(x,y),(x+w,y+h),(0,0,0),2)
    cv2.putText(roi,str(i),(x+w+10,y+h),0,0.3,(0,0,0))
    i+=1
mplot(roi)

In [ ]:
_, bw = cv2.threshold(roi, threshold, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
mplot(bw)

In [ ]:
a=np.array([[1,2],[2,3],[3,4],[5,2]])

In [ ]:
roi=thresh4
nroi=np.array([[255 if j>190 else j for j in roi[i]] for i in range(len(roi))],dtype=np.uint8)

In [ ]:
for i in words:
    mplot(i)

In [ ]:
len(words)